In [81]:
# Installation des dépendances
%pip install langchain pyttsx3 llama-cpp-python

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [82]:
#Installation des dépendances spécifiques au  RAG,
# ChromaDB (base vectorielle), pdfminer.six (lecture de PDF)  unstructured (extraction avancée) et sentence-transformers (pour transformer les textes en vecteurs)
%pip install chromadb unstructured pdfminer.six sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [83]:
import transformers
import numpy

print("Transformers et numpy sont bien importés !")
print("Version de numpy :", numpy.__version__)

Transformers et numpy sont bien importés !
Version de numpy : 1.26.4


In [84]:
# on vérifier quel le modèle est disponible
import os

model_path = "models/mistral-7b-instruct-v0.2.Q4_K_M.gguf" 

if os.path.exists(model_path):
    print("Modèle trouvé :", model_path)
else:
    print(" Modèle introuvable, vérifie le dossier models/")

Modèle trouvé : models/mistral-7b-instruct-v0.2.Q4_K_M.gguf


In [85]:
# On charge le modèle  en local avec llama-cpp-python
from langchain_community.llms import LlamaCpp 

llm = LlamaCpp(
    model_path=model_path, 
    temperature=0.7,
    max_tokens=900,
    top_p=0.9,
    verbose=True
)

print(" Modèle chargé en local !")

llama_model_load_from_file_impl: using device Metal (Apple M1 Pro) - 9779 MiB free
llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from models/mistral-7b-instruct-v0.2.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.r

 Modèle chargé en local !


Metal : EMBED_LIBRARY = 1 | CPU : NEON = 1 | ARM_FMA = 1 | FP16_VA = 1 | DOTPROD = 1 | LLAMAFILE = 1 | ACCELERATE = 1 | AARCH64_REPACK = 1 | 
Model metadata: {'general.quantization_version': '2', 'tokenizer.chat_template': "{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ message['content'] + eos_token}}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}", 'tokenizer.ggml.add_eos_token': 'false', 'tokenizer.ggml.add_bos_token': 'true', 'tokenizer.ggml.padding_token_id': '0', 'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'tokenizer.ggml.bos_token_id': '1', 'tokenizer.ggml.model': 'llama', 'llama.attention.head_count_kv': '8',

In [86]:
# On vérifie si le modèle  répond bien en français
response = llm.invoke("Peux-tu me parler des avancées en intelligence artificielle ?")
print("Chatbot :", response)

llama_perf_context_print:        load time =    4954.53 ms
llama_perf_context_print: prompt eval time =    4954.19 ms /    18 tokens (  275.23 ms per token,     3.63 tokens per second)
llama_perf_context_print:        eval time =   38698.55 ms /   493 runs   (   78.50 ms per token,    12.74 tokens per second)
llama_perf_context_print:       total time =   43917.30 ms /   511 tokens


Chatbot : 

Oui, je suis heureux de parler des avancées récentes en intelligence artificielle (IA).

1. DeepMind's AlphaZero: AlphaZero a appris à jouer au shogi, le go et l'échecs en seulement quelques heures, tout en battant les meilleurs programmes existants dans ces jeux. Cela a été possible grâce à AlphaZero's utilisation d'un apprentissage automatique renforcé (RL), sans aucune intervention humaine ni données externes utilisées pour l'apprentissage.

2. Google Brain's Neural Architecture Search (NAS): NAS est un algorithme de recherche automatisé d'architectures neurales efficaces et adaptatives à des jeux de données particuliers ou des applications spécifiques. Ce type de recherche automatisée peut être considérablement plus efficace que la conception manuelle traditionnelle d'architectures neurales, car elle explore un grand espace de recherche automatisé et identifie des architectures neurales adaptatives et efficaces qui peuvent être difficiles à concevoir par la méthode trad

In [87]:
# configure Langchain avec une mémoire pour suivre la conversation
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
chatbot = ConversationChain(llm=llm, memory=memory)

print("Chatbot prêt avec gestion du contexte intégrée !")

ggml_metal_free: deallocating


Chatbot prêt avec gestion du contexte intégrée !


In [88]:
#  verifier si les documents existent
import os

data_path = "data/"
text_file = f"{data_path}connaissances.txt"
pdf_file = f"{data_path}document.pdf"

if not os.path.exists(text_file):
    print(f" Le fichier {text_file} est introuvable ! Ajoute-le dans le dossier /data/")
if not os.path.exists(pdf_file):
    print(f" Le fichier {pdf_file} est introuvable ! Ajoute-le dans le dossier /data/")
else:
    print("Tous les fichiers sont bien présents.")

Tous les fichiers sont bien présents.


In [89]:
# tester si PyPDFLoader arrive à lire document.pdf
from langchain.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader("data/document.pdf")

try:
    docs = pdf_loader.load()
    print(f" Nombre de pages chargées : {len(docs)}")
    print(f" Extrait du document : {docs[0].page_content[:500]}")
except Exception as e:
    print(" Erreur lors de la lecture du PDF :", e)

 Nombre de pages chargées : 1
 Extrait du document : Dans	une	terrine,	mélange	la	crème,	la	
poudre	d’amandes	et	l’œuf.
	 Déroule	la	pâte	sablée	dans	un	plat,	
sur	le	papier	cuisson.	Puis	verses-y	ta	
préparation.	Préchauffe	le	four,	thermostat	7	
(200	°C).
	 Épluche	les	pommes	et	retire	le	
trognon.	Puis	découpe-les	en	minces	
quartiers.	Dispose-les	en	rond	sur	le	dessus	
du	fond	de	pâte.	Puis	parsème	quelques	
noisettes	de	beurre	et	saupoudre	de	sucre	
pour	faire	dorer	les	pommes	à	la	cuisson.
	 Mets	ta	tarte	au	four	pendant	
35	minutes.	Déguste


In [90]:
# tester si TextLoader arrive à lire connaissances.txt
from langchain.document_loaders import TextLoader

text_loader = TextLoader("data/connaissances.txt")

try:
    docs = text_loader.load()
    print(f" Nombre de lignes chargées : {len(docs)}")
    print(f" Extrait du texte : {docs[0].page_content[:500]}")
except Exception as e:
    print(" Erreur lors de la lecture du fichier texte :", e)

 Nombre de lignes chargées : 1
 Extrait du texte : # La tarte aux pommes : Histoire et Origines
La tarte aux pommes est l'un des desserts les plus populaires au monde. Son origine remonte au Moyen Âge, où les pommes étaient utilisées pour garnir des pâtes simples. En France, la tarte aux pommes est un dessert emblématique souvent accompagné d'une touche de cannelle ou de sucre vanillé.

# Les ingrédients essentiels
Une tarte aux pommes classique est composée de :
- Une pâte (brisée, sablée ou feuilletée)
- Des pommes (Golden, Granny Smith, Reine


In [91]:
# Charger et indexer les documents
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

data_path = "data/"

loaders = []
if os.path.exists(f"{data_path}connaissances.txt"):
    loaders.append(TextLoader(f"{data_path}connaissances.txt"))
if os.path.exists(f"{data_path}document.pdf"):
    loaders.append(PyPDFLoader(f"{data_path}document.pdf"))

if not loaders:
    print(" Aucun fichier trouvé, impossible de créer l'index.")
else:
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    documents = []
    for loader in loaders:
        documents.extend(loader.load())

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    documents = text_splitter.split_documents(documents)

    vectorstore = Chroma.from_documents(documents, embedding=embedding_model)

    print(" Base de connaissances indexée avec succès !")

 Base de connaissances indexée avec succès !


In [92]:
# utilise pyttsx3 pour lire les réponses du chatbot à voix haute
import pyttsx3

def text_to_speech(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

In [93]:
query = "Comment faire une tarte aux pommes ?"
docs = vectorstore.similarity_search(query, k=3)

print("Documents récupérés pour :", query)
for doc in docs:
    print(doc.page_content[:500]) 


Documents récupérés pour : Comment faire une tarte aux pommes ?
# Les différentes variantes
Il existe plusieurs façons de préparer la tarte aux pommes :
1. **La tarte normande** : Enrichie avec un appareil à base de crème, d'œufs et d'amandes.
2. **La tarte tatin** : Version caramélisée où les pommes sont cuites sous la pâte.
3. **La tarte fine aux pommes** : Pâte feuilletée très fine avec des tranches de pommes superposées.
# Les différentes variantes
Il existe plusieurs façons de préparer la tarte aux pommes :
1. **La tarte normande** : Enrichie avec un appareil à base de crème, d'œufs et d'amandes.
2. **La tarte tatin** : Version caramélisée où les pommes sont cuites sous la pâte.
3. **La tarte fine aux pommes** : Pâte feuilletée très fine avec des tranches de pommes superposées.
# Les différentes variantes
Il existe plusieurs façons de préparer la tarte aux pommes :
1. **La tarte normande** : Enrichie avec un appareil à base de crème, d'œufs et d'amandes.
2. **La tarte tatin** : Ve

In [94]:
#Crée une API Web , permet d’accéder à l’API depuis une interface web, et bark pour la synthèse vocale
%pip install flask flask-cors bark soundfile 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import numpy as np
import soundfile as sf
from bark import generate_audio, preload_models
import os
import sys
from threading import Thread
import torch

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}}, supports_credentials=True)

port = 7441  

original_torch_load = torch.load

def safe_load(*args, **kwargs):
    kwargs["weights_only"] = False 
    return original_torch_load(*args, **kwargs) 

torch.load = safe_load 

bark_model_path = "models/bark"
os.makedirs(bark_model_path, exist_ok=True)
os.environ["BARK_CACHE_DIR"] = bark_model_path

print("Téléchargement et chargement des modèles Bark...")
sys.stdout.flush()
preload_models()

def save_and_play_audio(audio_array, filename="response.wav"):
    print(f"Lecture de : {filename}")
    sys.stdout.flush()

    try:
        sf.write(filename, audio_array, samplerate=24000)
        if os.path.exists(filename) and os.path.getsize(filename) > 0:
            print(f"Fichier {filename} enregistré ({os.path.getsize(filename)} octets)")
            sys.stdout.flush()
            os.system(f"afplay {filename}")
        else:
            print("Erreur : Le fichier audio est vide ou n’a pas été créé.")
            sys.stdout.flush()
    except Exception as e:
        print(f"Erreur lors de la lecture du fichier audio : {e}")
        sys.stdout.flush()

@app.route("/chat", methods=["POST"])
def chat():
    user_input = request.json.get("message", "").strip()

    print(f"Message reçu : {user_input}")
    sys.stdout.flush()

    docs = vectorstore.similarity_search(user_input, k=3)
    context = "\n".join([doc.page_content for doc in docs])

    full_query = f"Réponds strictement en français en utilisant ces informations : {context}\nQuestion : {user_input}"
    print(f"Génération de la réponse...\n Contexte : {context}")
    sys.stdout.flush()

    response = chatbot.run(full_query)
    print(f"Réponse générée : {response}")
    sys.stdout.flush()

    print("Génération audio de la question...")
    sys.stdout.flush()
    
    try:
        audio_question = generate_audio(f"Vous avez demandé : {user_input}", history_prompt="v2/fr_speaker_2")
        save_and_play_audio(audio_question, "question.wav")

        print("Génération audio de la réponse...")
        sys.stdout.flush()

        audio_response = generate_audio(response, history_prompt="v2/fr_speaker_3")
        save_and_play_audio(audio_response, "response.wav")

    except Exception as e:
        print(f"Erreur lors de la génération audio avec Bark : {e}")
        sys.stdout.flush()

    return jsonify({"response": response})

print(f" Flask démarre sur le port {port}")
sys.stdout.flush()

def run():
    app.run(host="0.0.0.0", port=port, debug=True, use_reloader=False)

Thread(target=run).start()

Téléchargement et chargement des modèles Bark...


No GPU being used. Careful, inference might be very slow!


 Flask démarre sur le port 7441
 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:7441
 * Running on http://10.2.165.35:7441
Press CTRL+C to quit
127.0.0.1 - - [04/Mar/2025 16:36:29] "OPTIONS /chat HTTP/1.1" 200 -


Message reçu : Quelles sont les différentes variantes de la tarte aux pommes ?
Génération de la réponse...
 Contexte : # Les différentes variantes
Il existe plusieurs façons de préparer la tarte aux pommes :
1. **La tarte normande** : Enrichie avec un appareil à base de crème, d'œufs et d'amandes.
2. **La tarte tatin** : Version caramélisée où les pommes sont cuites sous la pâte.
3. **La tarte fine aux pommes** : Pâte feuilletée très fine avec des tranches de pommes superposées.
# Les différentes variantes
Il existe plusieurs façons de préparer la tarte aux pommes :
1. **La tarte normande** : Enrichie avec un appareil à base de crème, d'œufs et d'amandes.
2. **La tarte tatin** : Version caramélisée où les pommes sont cuites sous la pâte.
3. **La tarte fine aux pommes** : Pâte feuilletée très fine avec des tranches de pommes superposées.
# Les différentes variantes
Il existe plusieurs façons de préparer la tarte aux pommes :
1. **La tarte normande** : Enrichie avec un appareil à base de

...

100%|██████████| 17/17 [01:11<00:00,  4.19s/it]


Lecture de : question.wav
Fichier question.wav enregistré (313004 octets)
Génération audio de la réponse...


100%|██████████| 35/35 [02:29<00:00,  4.27s/it]


Lecture de : response.wav
Fichier response.wav enregistré (668204 octets)


127.0.0.1 - - [04/Mar/2025 16:42:31] "POST /chat HTTP/1.1" 200 -


In [96]:
html_code = """
<!DOCTYPE html>
<html>
<head>
    <title>Chatbot RAG</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }
        body {
            font-family: Arial, sans-serif;
            background-color: #f8f9fa;
            text-align: center;
            margin: 50px;
        }
        h2 {
            color: #343a40;
            margin-bottom: 20px;
        }
        input {
            width: 60%;
            padding: 10px;
            font-size: 16px;
            border: 2px solid #343a40;
            border-radius: 5px;
            margin-bottom: 10px;
        }
        button {
            background-color: #007bff;
            color: white;
            border: none;
            padding: 10px 20px;
            font-size: 16px;
            border-radius: 5px;
            cursor: pointer;
            transition: background 0.3s ease;
        }
        button:hover {
            background-color: #0056b3;
        }
        #chat_container {
            margin-top: 20px;
            width: 60%;
            display: inline-block;
            text-align: left;
        }
        .chat-message {
            padding: 15px;
            background-color: white;
            border-radius: 5px;
            border: 1px solid #ddd;
            margin-bottom: 10px;
            font-size: 16px;
        }
    </style>
    <script>
        let flaskPort = 7441; 

        async function sendMessage() {
            let userInput = document.getElementById("user_input").value;
            console.log(" Envoi de la requête avec le message :", userInput);

            try {
                let response = await fetch(`http://127.0.0.1:${flaskPort}/chat`, {
                    method: "POST",
                    headers: { "Content-Type": "application/json" },
                    body: JSON.stringify({ message: userInput })
                });

                console.log(" Réponse reçue de Flask :", response);
                
                let data = await response.json();
                let chatbotResponse = data.response;

                let chatContainer = document.getElementById("chat_container");
                let userMessage = document.createElement("p");
                userMessage.className = "chat-message";
                userMessage.innerHTML = "<strong>Vous :</strong> " + userInput;

                let botMessage = document.createElement("p");
                botMessage.className = "chat-message";
                botMessage.innerHTML = "<strong>Chatbot :</strong> " + chatbotResponse;

                chatContainer.appendChild(userMessage);
                chatContainer.appendChild(botMessage);

                document.getElementById("user_input").value = "";
            } catch (error) {
                console.error(" Erreur lors de la requête :", error);
            }
        }
    </script>
</head>
<body>
    <h2>Chatbot RAG</h2>
    <input type="text" id="user_input" placeholder="Écrivez votre question ici...">
    <button onclick="sendMessage()">Envoyer</button>
    <div id="chat_container"></div>
</body>
</html>
"""

with open("index.html", "w") as f:
    f.write(html_code)

print("Interface web `index.html` !")


Interface web `index.html` !
